# 色による動作決定デモンストレーション

## ライブラリおよびROSの設定

In [ ]:
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())

In [ ]:
# シミュレーションの場合はTrue, 実機の場合はFalseとする
sim_mode = True

In [ ]:
import os
if sim_mode:
    os.environ['ROS_MASTER_URI']='http://localhost:11311'
    os.environ['ROS_IP'] = 'localhost'
    os.environ['ROS_HOSTNAME'] = 'localhost'
else :
    robot_ip_addr = '133.15.97.82'
    host_ip_addr = '133.15.97.76'
    os.environ['ROS_MASTER_URI']='http://{}:11311'.format(robot_ip_addr)
    os.environ['ROS_IP'] = host_ip_addr
    os.environ['ROS_HOSTNAME'] = host_ip_addr

## ロボットインターフェースの作成

In [ ]:
ri = RobotInterface('robotinterface.yaml')

## システム動作

### モーションの作成

In [ ]:
if sim_mode:
    grasp_motion_list = [npa([0, 0,-2.3,-1.6,-0.9]), npa([0, -0.75,-2.3,-1.6,-0.9]), npa([0, -1.8, 0.3, 0,-0.9]), npa([0, -1.7,0.1, 0, 0]), npa([0, 0,-2.3,-1.6,0]),]
else :
    grasp_motion_list = [npa([0, 0,-2.3,-1.6,-0.9]), npa([0, -0.65,-2.3,-1.6,-0.9]), npa([0, -1.7,0.1, 0,-0.9]), npa([0, -1.7,0.1, 0, 0]), npa([0, 0,-2.3,-1.6,0]),]

putleft_motion_list = [npa([1, 0,-2.3,-1.6,0]), npa([1, -0.65,-2.3,-1.6,0]), npa([0.9, -0.65,-2.3,-1.6,-0.9]), npa([0.9, 0,-2.3,-1.6,-0.9])]
putright_motion_list = [npa([-1, 0,-2.3,-1.6,0]), npa([-1, -0.65,-2.3,-1.6,0]), npa([-1.1, -0.65,-2.3,-1.6,-0.9]), npa([-1.1, 0,-2.3,-1.6,-0.9])]


### 対象を把持

In [ ]:
speed = 1.5

for angle in grasp_motion_list:
    ri.sendAngleVector(angle,speed)
    ri.waitUntilFinish()

### センサデータ読み込み

In [ ]:
color_data = ri.waitNextData('COLOR_Sensor0', timeout=1.0, clear=True)
print(color_data)

### センサデータによる動作決定

In [ ]:
if color_data.b > 0.25:
    print("put left")
    for angle in putleft_motion_list:
        ri.sendAngleVector(angle,speed)
        ri.waitUntilFinish()
else :
    print("put right")
    for angle in putright_motion_list:
        ri.sendAngleVector(angle,speed)
        ri.waitUntilFinish()

ri.sendAngleVector(npa([0, 0,-2.3,-1.6,-0.9]),speed)
ri.waitUntilFinish()